# Código para incorporar palavras chaves ao dataset keywords

In [ ]:
# Importar o drive para importar o arquivo das pastas
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Utilização do spacy para PNL a partir de verbos e substantivos agrupando as palavras
Palavras iguais, mas em tempos diferentes

In [ ]:
import spacy
import os
from collections import Counter
import csv

In [ ]:
# Carregar o modelo de inglês
nlp = spacy.load("en_core_web_sm")

# Caminho do arquivo
FILE_US = "/content/drive/My Drive/TCC 2025/Base_de_dados/all_user_stories.txt"

# Percentual de histórias que serão processadas (0 a 100)
PERCENTUAL_LIDAS = 80

# Lista de palavras que devem ser ignoradas (estrutura da história de usuário)
PALAVRAS_IGNORADAS = {"as", "a", "i", "want", "so", "that"}

# Lista de verbos auxiliares válidos
AUXILIARES = {
    "be", "am", "is", "are", "was", "were",
    "do", "does", "did",
    "have", "has", "had",
    "will", "would", "shall", "should", "can", "could", "may", "might", "must", "need"
}

# Carregar histórias de usuário
def carregar_historias(filepath, percentual):
    if os.path.exists(filepath):
        with open(filepath, "r", encoding="utf-8") as arquivo:
            todas = []
            for linha in arquivo.readlines():
                linha = linha.strip()
                # # Divide na primeira vírgula e pega a parte após ela
                # if "," in linha:
                #     parte_util = linha.split(",", 1)[1].strip()
                #     todas.append(parte_util)
                # else:
                todas.append(linha)  # mantém linha inteira se não tiver vírgula

            qtd_lidas = max(1, int(len(todas) * percentual / 100))
            historias_lidas = todas[:qtd_lidas]
            historias_nao_lidas = todas[qtd_lidas:]
            return historias_lidas, historias_nao_lidas
    else:
        print(f"Arquivo {filepath} não encontrado.")
        return [], []


historias_lidas, historias_nao_lidas = carregar_historias(FILE_US, PERCENTUAL_LIDAS)


verbos_principais = []
substantivos = []
historias_com_verbos_substantivos = []

# Processar cada história de usuário individualmente
for historia in historias_lidas:
    doc = nlp(historia) # Monta o doc da frase, com as informações que o spacy processa de cada token


    verbos_historia_principal = []
    substantivos_historia = []

    for token in doc:
        token_lower = token.text.lower()

        # Ignorar palavras da estrutura padrão
        if token_lower in PALAVRAS_IGNORADAS:
            continue

        if token.pos_ == "VERB":
            verbo_base = token.lemma_.lower()

            # Se não for auxiliar válido, considera como verbo principal
            if verbo_base not in AUXILIARES:
                verbos_principais.append(verbo_base)
                verbos_historia_principal.append(verbo_base)

        elif token.pos_ == "NOUN":
            substantivo_base = token.lemma_.lower()
            substantivos_historia.append(substantivo_base)
            substantivos.append(substantivo_base)

    historias_com_verbos_substantivos.append({
        "historia": historia,
        "verbos_principais": verbos_historia_principal,
        "substantivos": substantivos_historia
    })

# Contar a frequência e ordenar por frequência decrescente
contador_verbos_principais = Counter(verbos_principais).most_common()
contador_substantivos = Counter(substantivos).most_common()

# Caminho do arquivo de saída
CSV_SAIDA = "/content/drive/My Drive/TCC 2025/Base_de_dados/saida_palavras_sprint_final.csv"

# Combinar os dados em uma lista de tuplas (palavra, frequência, tipo)
dados_csv = []

for verbo, count in contador_verbos_principais:
    dados_csv.append((verbo, count, "V"))

for substantivo, count in contador_substantivos:
    dados_csv.append((substantivo, count, "N"))

# Escrever no arquivo CSV
with open(CSV_SAIDA, mode="w", newline="", encoding="utf-8") as arquivo_csv:
    escritor = csv.writer(arquivo_csv)
    escritor.writerow(["palavra", "frequência", "tipo"])  # cabeçalho
    escritor.writerows(dados_csv)

print(f"\nArquivo CSV salvo em: {CSV_SAIDA}")





Arquivo CSV salvo em: /content/drive/My Drive/TCC 2025/Base_de_dados/saida_palavras_agrupadas_sprint_final.csv


In [ ]:

# Caminho do arquivo de saída TXT para as histórias não utilizadas
TXT_NAO_LIDAS = "/content/drive/My Drive/TCC 2025/Base_de_dados/historias_nao_utilizadas_sprint_final.txt"

# Escrever as histórias não utilizadas no arquivo TXT
if historias_nao_lidas:
    with open(TXT_NAO_LIDAS, mode="w", encoding="utf-8") as arquivo_txt:
        for historia in historias_nao_lidas:
            arquivo_txt.write(historia + "\n")
    print(f"Arquivo TXT com as histórias não utilizadas salvo em: {TXT_NAO_LIDAS}")
else:
    print("Não há histórias de usuário restantes para salvar.")

Arquivo TXT com as histórias não utilizadas salvo em: /content/drive/My Drive/TCC 2025/Base_de_dados/historias_nao_utilizadas_sprint_final.txt


In [ ]:
# Caminho do arquivo de saída TXT para as histórias lidas
TXT_LIDAS = "/content/drive/My Drive/TCC 2025/Base_de_dados/historias_utilizadas_sprint_final.txt"

# Escrever as histórias lidas no arquivo TXT
if historias_lidas:
    with open(TXT_LIDAS, mode="w", encoding="utf-8") as arquivo_txt_lidas:
        for historia in historias_lidas:
            arquivo_txt_lidas.write(historia + "\n")
    print(f"Arquivo TXT com as histórias utilizadas salvo em: {TXT_LIDAS}")
else:
    print("Não houve histórias de usuário lidas para salvar.")

Arquivo TXT com as histórias utilizadas salvo em: /content/drive/My Drive/TCC 2025/Base_de_dados/historias_utilizadas_sprint_final.txt
